In [1]:
#Imports
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import pickle
import tqdm
import line_profiler
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder
import math

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [3]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

In [4]:
#pickle Load results

resultspicklefile = open( 'results-df.npy', 'rb')
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
print(resultsdf.head())
split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "track"])

     race_id       date     track  dist        grade  place      dog_id box  \
0  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    1.0  1296100098   2   
1  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    2.0  1177320005   3   
2  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    3.0  1549980026   5   
3  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    4.0  1474670055   7   
4  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    5.0   913850094   8   

  split_times split_margins run_time weigth margin  start_price  \
0        6.73          0.00    26.17   30.2      0         10.6   
1        6.87          0.14    26.18   29.3   0.20          6.1   
2        6.99          0.26    26.26   29.4   1.24         11.9   
3        6.88          0.15    26.36   32.8   2.79          3.3   
4        6.91          0.18    26.42   32.4   3.60         16.0   

         dog_name  
0  SWIFT OPERATOR  
1   LAST PRINCESS  
2  DAINTREE PRINZ  
3    TIC TOC TOBY  
4   RACE ROQUETTE  


In [47]:
betfair_df = pickle.load(open('df-betfairSP.npy', 'rb'))
betfair_df['dateF'] = pd.to_datetime(betfair_df.EVENT_DT, dayfirst=True).dt.date
betfair_df['dog_name'] = betfair_df.dog.str[1:].str.upper()
resultsdf['dateF']= pd.to_datetime(resultsdf.date).dt.date

In [11]:
resultsdf.head()

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin,start_price,dog_name,track_id,dateF
0,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0,10.6,SWIFT OPERATOR,Ball-450,2017-04-01
1,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20,6.1,LAST PRINCESS,Ball-450,2017-04-01
2,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24,11.9,DAINTREE PRINZ,Ball-450,2017-04-01
3,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79,3.3,TIC TOC TOBY,Ball-450,2017-04-01
4,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60,16.0,RACE ROQUETTE,Ball-450,2017-04-01


In [48]:
def input_layer_gen(form):
    race_forms = form.groupby(["race_id"])
    for i,j in race_forms:
        print(i,j)
        break

In [8]:
resultsdf[resultsdf['start_price']>0]

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin,start_price,dog_name,track_id,dateF
0,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0,10.6,SWIFT OPERATOR,Ball-450,2017-04-01
1,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20,6.1,LAST PRINCESS,Ball-450,2017-04-01
2,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24,11.9,DAINTREE PRINZ,Ball-450,2017-04-01
3,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79,3.3,TIC TOC TOBY,Ball-450,2017-04-01
4,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60,16.0,RACE ROQUETTE,Ball-450,2017-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539357,489942463,31 Oct 19,Warragul,400.0,Grade 5 T3,4.0,124928859,6,8.56,0.16,23.35,28.3,5.63,10.9,BOMBER'S BABY,Warr-400,2019-10-31
539358,489942463,31 Oct 19,Warragul,400.0,Grade 5 T3,5.0,1549980001,4,8.66,0.26,23.49,34.3,7.66,12.8,ALADDIN PRINCE,Warr-400,2019-10-31
539359,489942463,31 Oct 19,Warragul,400.0,Grade 5 T3,6.0,1206250131,2,8.67,0.27,23.55,39.8,8.46,13.4,PHILO BALE,Warr-400,2019-10-31
539360,489942463,31 Oct 19,Warragul,400.0,Grade 5 T3,7.0,2056920061,1,8.93,0.53,23.97,37.7,14.46,7.9,GENE MAN,Warr-400,2019-10-31


In [51]:
# pickle Load results
resultspicklefile = open("results-df.npy", "rb")
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
resultsdf.head()

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(resultsdf[['track']])
#enc = OneHotEncoder( categories=tracks)
print(resultsdf.shape)
print(len(transformed))
resultsdf["tracksOnehot"] = transformed.tolist()

split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf.split_margins = resultsdf.split_margins.astype(float)
resultsdf["track_id"] = resultsdf.apply(
    lambda s: track_id_gen(s["dist"], s["track"]), axis=1
)

resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "track"])

resultsdf_merged["dateF1"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["run_time"]) / pd.to_numeric(
    resultsdf_merged["dist_x"]
)

resultsdf_merged["split_speed"] = pd.to_numeric(
    resultsdf_merged["split_times"]
) / pd.to_numeric(resultsdf_merged["split_dist_estim"])

resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["box"])
resultsdf_merged["margin"] = pd.to_numeric(resultsdf_merged["margin"])
resultsdf_merged["weigth"] = pd.to_numeric(resultsdf_merged["weigth"])
resultsdf_merged["dateF"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y").dt.date

resultsdf_merged = pd.merge(
    resultsdf_merged, betfair_df, how="left", on=["dateF", "dog_name"]
)

form = resultsdf_merged.sort_values("dateF", ascending=False)

with open("results-df-merged-prices.npy", "wb") as fp:   #Pickling
    
    pickle.dump(form, fp)

    
print(form.head())

form = form.sort_values(['dateF'])
race_forms = form.groupby(["race_id"])
dog_forms = form.groupby(["dog_id"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details = []
dog_stats_df = []
resultsdf_merged.head(50)
n = 0
for i, j in tqdm.tqdm(race_forms):
    try:
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        dogs = j.dog_id
        race_id = i
        race_dist = j.dist_x.iloc[1]
        trackOHE = j.tracksOnehot.iloc[1]
        dog_info = []
        num_features = 16
        

        #Starts off with blank outfits of what we need to fill, are then filled using box pos as indexer
        blank_stats = [-1]*num_features
        blank_race = [blank_stats]*8
        blank_classes = [8]*8
        blank_margins = [80]*8
        blank_sp = [0]*8
        blank_bsp = [0]*8
        blank_dogname = ["blank"]*8
        if len(dogs)>8:
            continue
        i = 0
        for dog in dogs:
            
            # print(f"dog id = {dog}")
            prev_form = dog_forms.get_group(dog)
            prev_form = prev_form[(prev_form["dateF"] < race_date)]
            prev_races = prev_form.shape[0]
            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            if prev_races:
                dog_margin_last = prev_form.margin.iloc[0]
                dog_speed_last = prev_form.speed.iloc[0]
            else:
                dog_margin_last = -1
                dog_speed_last = -1
            # print(dog_margin_last)
            dog_race_count = prev_form.shape[0]
            dog_wins = prev_form[prev_form["place"] == 1].shape[0]
            dog_places = prev_form[prev_form["place"] <= 3].shape[0]
            dog_box = j.box.iloc[i]
            dog_weight = j.weigth.iloc[i] # weight not in forms for prediction going to use last weight
            stats = [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                    ]
            #stats = [-1 if math.isnan(x) else x for x in stats]
            dog_stats = (
                dog,
                stats,
                j.place.iloc[i],
                j.start_price.iloc[i],
                j.margin.iloc[i],
                j.BSP.iloc[i],
            )

            #Fill in the blank outputs
            idx = dog_box-1
            blank_race[idx] = stats
            blank_classes[idx] = j.place.iloc[i]
            blank_bsp[idx] = j.BSP.iloc[i]
            blank_margins[idx] = j.margin.iloc[i]
            blank_sp[idx] = j.start_price.iloc[i]
            blank_dogname[idx] = j.dog_name.iloc[i]




            # dog_info.append(dog_stats)
            # dog_stats_df.append([dog,
            #         race_id,
            #         race_date,
            #         trackOHE,
            #         race_dist,
            #         stats,
            #         j.place.iloc[i],
            #         j.start_price.iloc[i],
            #         j.margin.iloc[i],
            #         j.BSP.iloc[i]])
            #print(f"{dog_stats_df=}")
            i = i + 1
        
        

        dog_info.sort(key=itemgetter(0))
        dog_stats_df.append(dog_info)
        input_layer = (
            trackOHE
            + [race_dist]
            + [item for sublist in blank_race for item in sublist]
        )
        
        # if len(dogs) > 8:
        #     print(f"{race_id=}")
        #     print(f"{race_date=}")
        #     print(f"{j.track_x.iloc[1]=}")
        classes = blank_classes
        prices = blank_sp
        margins = blank_margins
        betfairSP = blank_bsp
        dogname = blank_dogname
        race_input_layer.append(input_layer)
        race_classes.append(classes)
        full_details.append(
            (race_id, input_layer, classes, prices,  margins, betfairSP, dogname)
        )

        
    except Exception as e:
        print(f"{e=}")
    # n += 1
    # if n >10:
    #     break

(537871, 15)
537871
          race_id       date          track  dist_x         grade  place  \
290645  828638479  28 Sep 22  Meadows (MEP)   525.0       Grade 5    1.0   
295411  828638469  28 Sep 22  Meadows (MEP)   600.0     Mixed 6/7    2.0   
295403  828638476  28 Sep 22  Meadows (MEP)   600.0  Grade 5 Heat    2.0   
295404  828638476  28 Sep 22  Meadows (MEP)   600.0  Grade 5 Heat    3.0   
295405  828638476  28 Sep 22  Meadows (MEP)   600.0  Grade 5 Heat    4.0   

           dog_id  box split_times  split_margins  ...  PPMAX  PPMIN  IPMAX  \
290645  440000255    8        5.21           0.01  ...    NaN    NaN    NaN   
295411  617355258    1        9.24           0.12  ...    NaN    NaN    NaN   
295403  341836817    2        9.34           0.33  ...    NaN    NaN    NaN   
295404  595041712    5        9.10           0.09  ...    NaN    NaN    NaN   
295405  462264686    4        9.29           0.28  ...    NaN    NaN    NaN   

        IPMIN MORNINGTRADEDVOL PPTRADEDVOL IPTRA

 67%|██████▋   | 49037/72724 [15:04<05:53, 66.95it/s] 

e=IndexError('single positional indexer is out-of-bounds')


 82%|████████▏ | 59290/72724 [18:01<03:46, 59.43it/s] 

e=IndexError('single positional indexer is out-of-bounds')


100%|██████████| 72724/72724 [21:51<00:00, 55.43it/s]


full_details

In [2]:
testdf = pd.DataFrame(race_input_layer)
print(testdf.shape)
no_nans = testdf.dropna()
print(no_nans.shape)

NameError: name 'pd' is not defined

In [52]:
with open("total_list_new_box.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details, fp)

In [14]:
with open("dog_stats_df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(dfx, fp)

In [27]:
file = open( "total_list.npy", 'rb')
data = pickle.load(file)
raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
betfairSP

FileNotFoundError: [Errno 2] No such file or directory: 'total_list.npy'

In [ ]:
bfSPdf = pd.DataFrame(betfairSP)
bfSPdf

,0,1,2,3,4,5,6,7
0,120.000000,12.000000,440.000000,5.599896,5.800000,3.000000,NaN,50.000000
1,21.197895,1.881559,23.296934,6.400000,6.400000,56.414166,20.262804,20.498423
2,7.400000,91.039696,30.451385,1.799578,6.733082,20.000000,14.464201,26.508447
3,NaN,18.000000,45.760471,NaN,4.832985,3.035935,25.787296,7.262075
4,3.400000,16.969517,11.408280,32.000000,2.829145,9.931054,27.000000,26.000000
...,...,...,...,...,...,...,...,...
71763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
